# Air flight cancellation data preparation
## by Valeriya Petrenko

## Preliminary Wrangling

> The data comes from the U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics (BTS), which  tracks the on-time performance of domestic flights operated by large air carriers.

> The data contains flight delays and cancellations. We'll focus only on flight cancellation part.

> Additional data was obtained from: http://stat-computing.org/dataexpo/2009/supplemental-data.html
  * information about carriers: carriers.csv
  * information about airports: airports.csv
  

In [1]:
from utils import *

### Get cancellation data for (1987-2009) period

In [2]:
DATA_PATH = "./data/data/"

In [3]:
loader = AirflightDelayDataLoader(DATA_PATH)
years = range(1987, 2009)
cols = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', # date columns
        'DepTime', 'CRSDepTime',                    # departure
        'ArrTime', 'CRSArrTime',                    # arrival
        'UniqueCarrier', 'FlightNum',               # air carrier related
        'Origin', 'Dest',                           # origin/destination
        'Cancelled', 'Diverted', 'CancellationCode' # cancellation
       ]
# using all these columns results in 10Gb data usage in memory
# and requires 20Gb of RAM during processing
# missing values: total rows = 123534969
# DepTime: 2302136 (1.8% of total data)
# ArrTime: 2584478 (2.0% of total data)
# CancellationCode: 83844440 (68% of total data) 

In [3]:
# instead, let's just focus on a reduced set comprising of whether a flight was cancelled
# at destinations

In [4]:
%%time
cancel_cols = ['Year', 'Month', 'UniqueCarrier', 'Dest', 'Cancelled']
cancellation_fname = os.path.join(DATA_PATH, "df_cancellation_1987-2008.pkl")
if not os.path.exists(cancellation_fname):
    df = loader.get_data(years, cancel_cols)
    df.to_pickle(cancellation_fname)
else:
    df = pd.read_pickle(cancellation_fname)

CPU times: user 5.11 s, sys: 2.3 s, total: 7.42 s
Wall time: 13.5 s


In [5]:
df.dtypes, df.shape

(Year              int16
 Month              int8
 UniqueCarrier    object
 Dest             object
 Cancelled          int8
 dtype: object, (123534969, 5))

In [6]:
memory_usage(df)

'Memory usage: 3298.7 Mb'

In [7]:
# convert object types to category
df.UniqueCarrier = df.UniqueCarrier.astype('category')
df.Dest = df.Dest.astype('category')

In [8]:
df.head()

,Year,Month,UniqueCarrier,Dest,Cancelled
0,1987,10,PS,SFO,0
1,1987,10,PS,SFO,0
2,1987,10,PS,SFO,0
3,1987,10,PS,SFO,0
4,1987,10,PS,SFO,0


In [9]:
# since 1987 starts with month 10, which means we have data for 3 months only, drop this year
df = df[df.Year > 1987]

In [10]:
# combine all flights by Year, Month, UniqueCarrier and Dest
index_cols = ['Year', 'Month', 'UniqueCarrier', 'Dest']
s_cancel = df.groupby(index_cols)['Cancelled'].sum()
s_tot = df.groupby(index_cols)['Cancelled'].count()
df1 = pd.concat([s_tot, s_cancel], axis=1)
df1.columns = ["Flights", "Cancelled"]

In [11]:
df1 = df1.reset_index()
df1.head()

,Year,Month,UniqueCarrier,Dest,Flights,Cancelled
0,1988,1,AA,ABQ,264,11.0
1,1988,1,AA,ALB,117,2.0
2,1988,1,AA,AMA,117,7.0
3,1988,1,AA,ATL,531,39.0
4,1988,1,AA,AUS,324,19.0


In [12]:
df1.tail()

,Year,Month,UniqueCarrier,Dest,Flights,Cancelled
261244,2008,12,YV,TEX,28,6.0
261245,2008,12,YV,TUS,275,1.0
261246,2008,12,YV,TVC,48,3.0
261247,2008,12,YV,TYS,71,4.0
261248,2008,12,YV,YUM,138,2.0


In [13]:
df1.dtypes

Year                int64
Month               int64
UniqueCarrier    category
Dest             category
Flights             int64
Cancelled         float64
dtype: object

In [14]:
# all rows have data
df1.isnull().sum()

Year             0
Month            0
UniqueCarrier    0
Dest             0
Flights          0
Cancelled        0
dtype: int64

### Now load data for 06/2003 to 02/2019 period

In [15]:
df2 = pd.read_csv("data/data/062003_022019_airline_delay_causes.csv")
df2.columns = [x.strip() for x in df2.columns]
# last column has no data
df2 = df2.drop(labels=df2.columns[-1], axis=1)
columns = ["year", "month", "carrier", "airport", "arr_flights", "arr_cancelled"]
df2 = df2[columns]
df2.head()

,year,month,carrier,airport,arr_flights,arr_cancelled
0,2003,6,AA,ABQ,307.0,1.0
1,2003,6,AA,ANC,90.0,0.0
2,2003,6,AA,ATL,752.0,5.0
3,2003,6,AA,AUS,842.0,9.0
4,2003,6,AA,BDL,383.0,0.0


In [16]:
# this dataset has some rows without data, drop them
df2.isnull().sum()

year               0
month              0
carrier            0
airport            0
arr_flights      362
arr_cancelled    362
dtype: int64

In [17]:
df2 = df2.dropna()
df2.head()

,year,month,carrier,airport,arr_flights,arr_cancelled
0,2003,6,AA,ABQ,307.0,1.0
1,2003,6,AA,ANC,90.0,0.0
2,2003,6,AA,ATL,752.0,5.0
3,2003,6,AA,AUS,842.0,9.0
4,2003,6,AA,BDL,383.0,0.0


In [18]:
# convert carrier and airport to categorical variables
df2.carrier = df2.carrier.astype('category')
df2.airport = df2.airport.astype('category')
df2.arr_flights = df2.arr_flights.astype(np.int64)

In [19]:
df2.dtypes

year                int64
month               int64
carrier          category
airport          category
arr_flights         int64
arr_cancelled     float64
dtype: object

In [20]:
# since data for 2019 and 2003 is not for whole year, drop those years
df2 = df2[(df2['year'] > 2003) & (df2['year'] < 2019)]
df2.shape

(243749, 6)

### Since two datasets have common periods we can validate data in this period

In [21]:
df1[df1['Year'] == 2004].head()

,Year,Month,UniqueCarrier,Dest,Flights,Cancelled
171105,2004,1,AA,ABQ,275,1.0
171106,2004,1,AA,ATL,704,13.0
171107,2004,1,AA,AUS,793,17.0
171108,2004,1,AA,BDL,368,13.0
171109,2004,1,AA,BHM,93,0.0


In [22]:
df2[df2['year'] == 2004].head()

,year,month,carrier,airport,arr_flights,arr_cancelled
8763,2004,1,AA,ABQ,275,1.0
8764,2004,1,AA,ATL,704,13.0
8765,2004,1,AA,AUS,793,17.0
8766,2004,1,AA,BDL,368,13.0
8767,2004,1,AA,BHM,93,0.0


In [23]:
# assert we have the same results from two datasets
assert df1[df1.Year == 2004].shape == df2[df2.year == 2004].shape

In [24]:
# since 2004 year is in both, remove it from the first table
df1 = df1[df1.Year < 2004]
# to concatenate both datasets the column names should be the same
df2.columns = df1.columns
# combine two datasets into one
dft = pd.concat([df1, df2])

In [25]:
dft.head()

,Year,Month,UniqueCarrier,Dest,Flights,Cancelled
0,1988,1,AA,ABQ,264,11.0
1,1988,1,AA,ALB,117,2.0
2,1988,1,AA,AMA,117,7.0
3,1988,1,AA,ATL,531,39.0
4,1988,1,AA,AUS,324,19.0


In [26]:
dft.tail()

,Year,Month,UniqueCarrier,Dest,Flights,Cancelled
252859,2018,12,YX,TLH,47,1.0
252860,2018,12,YX,TUL,39,0.0
252861,2018,12,YX,TYS,18,0.0
252862,2018,12,YX,VPS,31,0.0
252863,2018,12,YX,XNA,76,0.0


In [27]:
dft.shape

(414854, 6)

In [28]:
# finally save to pickle file
fname = os.path.join(DATA_PATH, "df_cancellation_1988-2018.pkl")
dft.to_pickle(fname)